In [61]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt
import imutils
from imutils import perspective
import math
import time
import sys

In [62]:
img1 = cv2.imread('../imageTemp/fea1.png')
img2 = cv2.imread('../imageTemp/fea2.png')

img1orig = img1.copy()
img2orig = img2.copy()
img1Gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2Gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)


resOutH = 512

In [63]:
MIN_MATCH_COUNT = 10

# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1Gray,None)
kp2, des2 = sift.detectAndCompute(img2Gray,None)

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(des1,des2,k=2)

# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)
        
if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()

    h,w = img1Gray.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)

    img2 = cv2.polylines(img2,[np.int32(dst)], True, (0,255,255), 3, cv2.LINE_AA)

else:
    print ("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
    matchesMask = None
    
draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)

img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)

img1 = imutils.resize(img1, height = resOutH)
img2 = imutils.resize(img2, height = resOutH)
img3 = imutils.resize(img3, height = resOutH)

cv2.imshow('img1orig', img1orig)
cv2.imshow('img2orig', img2orig)
cv2.imshow('img2', img2)
cv2.imshow('output', img3)

In [64]:
while(True):
    getKey = cv2.waitKey(10) & 0xFF
    if getKey is ord('q'):
        cv2.destroyAllWindows()
        break
    if getKey is ord('s'):
        fileName = "../imageCapture/img2" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
        cv2.imwrite(fileName, img2, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
        fileName = "../imageCapture/img3" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
        cv2.imwrite(fileName, img3, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
        print('file saved')

file saved
